# Box Office Movie Franchise Predictor - Cleaning

Before embarking on this, a lot of time was spent searching for these movies. Because the number of movies that belong in a sequel fashion is limited, the scope of the study also includes movies that belong to a franchise and universe. A franchise example is James Bond and one that belongs to a universe is the Marvel Cinematic Universe (MCU). 

The time sinks in gathering and cleaning the data were:

1. Release Dates - Some formats were incomplete and had to be researched in either IMDB and cross-referenced with Wikipedia


2. Budgets & Currency - Missing budgets and foreign currency had to be converted accordingly to USD based on the rough exchange rate on the day of release


3. Correcting dates that were 1968 and earlier. Python parses the dates in the csv file a little strange and gave dates that are in 2068 for example. This is corrected.


## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

## Load Data File(s)

In [2]:
data_path = './data/'
file_name = 'movies_with_sequels_final.csv'

df = pd.read_csv(data_path + file_name)

## Data Wrangling & Cleaning

In [3]:
df.head(6)

,Title,url,IMDB Score,Metacritic,Runtime (mins),Budget,Opening Weekend,Gross USA,Gross World,Release Date,Rating,Genres,Country
0,Spider-Man (2002),http://www.imdb.com/title/tt0145487/,7.3,73.0,121,"$139,000,000",114844116,407022860,825025036,3-May-02,PG-13,Action Adventure Sci-Fi,USA
1,Spider-Man 2 (2004),http://www.imdb.com/title/tt0316654/,7.3,83.0,127,"$200,000,000",88156227,373585825,788976453,30-Jun-04,PG-13,Action Adventure Sci-Fi,USA
2,The Matrix (1999),http://www.imdb.com/title/tt0133093/,8.7,73.0,136,"$63,000,000",27788331,171479930,465343787,31-Mar-99,R,Action Sci-Fi,USA
3,The Matrix Reloaded (2003),http://www.imdb.com/title/tt0234215/,7.2,62.0,138,"$150,000,000",91774413,281576461,741846459,15-May-03,R,Action Sci-Fi,USA
4,The Lord of the Rings: The Fellowship of the R...,http://www.imdb.com/title/tt0120737/,8.8,92.0,178,"$93,000,000",47211490,315544750,887832826,19-Dec-01,PG-13,Action Adventure Drama,New Zealand
5,The Lord of the Rings: The Two Towers (2002),http://www.imdb.com/title/tt0167261/,8.7,87.0,179,"$94,000,000",62007528,342551365,951208089,18-Dec-02,PG-13,Adventure Drama Fantasy,New Zealand


### A Note about the Dataset

One very important note on how the csv file is arranged. The entire dataset as one can see from the above, i.e. an original movie or first movie and its corresponding sequel is arranged such that 

1. Movie 1
2. Sequel of Movie 1
3. Another film
4. Sequel of Another Film
5. ...
6. ...

The first 6 rows of the Pandas data frame (above), the movie is arranged as discussed. However, there was a Spider-Man 3 but it is not included here, i.e. index 2. The structure here has to be in even numbers and trilogies are not considered here for the time being. If the dataset is to be absent of leaky variables, the structure of observations must be in even numbers. That way, after cleaning, we can collect every other row as a target without the fear of a leaky variable.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078 entries, 0 to 1077
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            1078 non-null   object 
 1   url              1078 non-null   object 
 2   IMDB Score       1078 non-null   float64
 3   Metacritic       973 non-null    float64
 4   Runtime (mins)   1078 non-null   int64  
 5   Budget           1078 non-null   object 
 6   Opening Weekend  952 non-null    object 
 7   Gross USA        1005 non-null   object 
 8   Gross World      1078 non-null   object 
 9   Release Date     1078 non-null   object 
 10  Rating           1078 non-null   object 
 11  Genres           1078 non-null   object 
 12  Country          1078 non-null   object 
dtypes: float64(2), int64(1), object(10)
memory usage: 109.6+ KB


From the info table above, there are no missing values. This is incorrect. There are missing values but are stated as 'None' in their respective columns and that is why an entire column gets converted to an object. This was done initially in the webscrapping portion of the work. It has been corrected to be replaced with np.nan. Nonetheless, the change will be made here.

### Replace 'None' with NaN

In [5]:
def replace_none_w_nan(data_frame):
    
    column_names = list(data_frame.columns)
    
    for col in column_names:
        data_frame[col].replace('None',np.nan, inplace=True)
        
    return data_frame


In [6]:
df = replace_none_w_nan(df)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078 entries, 0 to 1077
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            1078 non-null   object 
 1   url              1078 non-null   object 
 2   IMDB Score       1078 non-null   float64
 3   Metacritic       973 non-null    float64
 4   Runtime (mins)   1078 non-null   int64  
 5   Budget           1078 non-null   object 
 6   Opening Weekend  952 non-null    object 
 7   Gross USA        1005 non-null   object 
 8   Gross World      1078 non-null   object 
 9   Release Date     1078 non-null   object 
 10  Rating           1078 non-null   object 
 11  Genres           1078 non-null   object 
 12  Country          1078 non-null   object 
dtypes: float64(2), int64(1), object(10)
memory usage: 109.6+ KB


This is much better. There are two missing budget entries which can be fixed. Too many missing values for

- Metacritic Score
- Opening Weekend
- Gross USA

First let's drop the Opening Weekend and Gross USA columns as there is too much variability in those. The Opening Weekend metric was not really tallied only until the late 90's. Let's see what we can do with the Metacritic and two missing Budgets. The missing budgets are from the movies Homeward Bound 1 and 2. But first, we need to clean them. 

### Clean-Up Budget

In [7]:
def money_to_float(money_str):
    
    money_str = str(money_str)
    money_str = money_str.strip()
    
    return float(money_str.replace('$','').replace(',',''))


In [8]:
# Clean money and convert to float
df['Budget'] = df['Budget'].apply(money_to_float)
df['Opening Weekend'] = df['Opening Weekend'].apply(money_to_float)
df['Gross USA'] = df['Gross USA'].apply(money_to_float)
df['Gross World'] = df['Gross World'].apply(money_to_float)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078 entries, 0 to 1077
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            1078 non-null   object 
 1   url              1078 non-null   object 
 2   IMDB Score       1078 non-null   float64
 3   Metacritic       973 non-null    float64
 4   Runtime (mins)   1078 non-null   int64  
 5   Budget           1078 non-null   float64
 6   Opening Weekend  952 non-null    float64
 7   Gross USA        1005 non-null   float64
 8   Gross World      1078 non-null   float64
 9   Release Date     1078 non-null   object 
 10  Rating           1078 non-null   object 
 11  Genres           1078 non-null   object 
 12  Country          1078 non-null   object 
dtypes: float64(6), int64(1), object(6)
memory usage: 109.6+ KB


In [10]:
df.loc[346]

Title                         Planet of the Apes (1968)
url                http://www.imdb.com/title/tt0063442/
IMDB Score                                            8
Metacritic                                           79
Runtime (mins)                                      112
Budget                                          5.8e+06
Opening Weekend                                     NaN
Gross USA                                   3.25896e+07
Gross World                                  3.2596e+07
Release Date                                   3-Apr-68
Rating                                                G
Genres                                 Adventure Sci-Fi
Country                                             USA
Name: 346, dtype: object

### Clean & Fix Release Dates

There is an apparent problem with Python when handling dates from 1968 or 1969. Apparently, if the year is 3-Nov-68, it converts it to 2068-11-03. Hence, a fix is needed for this.

In [11]:
# Convert Release Date to proper date
# This is an example showing the problem
df['Release Date'] = pd.to_datetime(df['Release Date']) #,format='%d-%b-%y')

df.loc[346]

Title                         Planet of the Apes (1968)
url                http://www.imdb.com/title/tt0063442/
IMDB Score                                            8
Metacritic                                           79
Runtime (mins)                                      112
Budget                                          5.8e+06
Opening Weekend                                     NaN
Gross USA                                   3.25896e+07
Gross World                                  3.2596e+07
Release Date                        2068-04-03 00:00:00
Rating                                                G
Genres                                 Adventure Sci-Fi
Country                                             USA
Name: 346, dtype: object

In [12]:
def fix_dates(rel_date):
    
    # Dates should not go this far
    cut_off = 2030
    
    if rel_date.year > cut_off:    
        year = rel_date.year - 100
    else:
        year = rel_date.year
    
    return datetime.date(year,rel_date.month,rel_date.day)

In [13]:
df['Release Date'] = df['Release Date'].apply(fix_dates)

In [14]:
df.head(6)

,Title,url,IMDB Score,Metacritic,Runtime (mins),Budget,Opening Weekend,Gross USA,Gross World,Release Date,Rating,Genres,Country
0,Spider-Man (2002),http://www.imdb.com/title/tt0145487/,7.3,73.0,121,139000000.0,114844116.0,407022860.0,825025036.0,2002-05-03,PG-13,Action Adventure Sci-Fi,USA
1,Spider-Man 2 (2004),http://www.imdb.com/title/tt0316654/,7.3,83.0,127,200000000.0,88156227.0,373585825.0,788976453.0,2004-06-30,PG-13,Action Adventure Sci-Fi,USA
2,The Matrix (1999),http://www.imdb.com/title/tt0133093/,8.7,73.0,136,63000000.0,27788331.0,171479930.0,465343787.0,1999-03-31,R,Action Sci-Fi,USA
3,The Matrix Reloaded (2003),http://www.imdb.com/title/tt0234215/,7.2,62.0,138,150000000.0,91774413.0,281576461.0,741846459.0,2003-05-15,R,Action Sci-Fi,USA
4,The Lord of the Rings: The Fellowship of the R...,http://www.imdb.com/title/tt0120737/,8.8,92.0,178,93000000.0,47211490.0,315544750.0,887832826.0,2001-12-19,PG-13,Action Adventure Drama,New Zealand
5,The Lord of the Rings: The Two Towers (2002),http://www.imdb.com/title/tt0167261/,8.7,87.0,179,94000000.0,62007528.0,342551365.0,951208089.0,2002-12-18,PG-13,Adventure Drama Fantasy,New Zealand


### Write to a New File

In [15]:
# Dump to new csv
file_name_cleaned = 'movies_with_sequels_final_cleaned.csv'

df.to_csv(data_path + file_name_cleaned, index=False)


## Splitting the Dataset to 'Original' and 'Sequel'

In [16]:
# Dump original and sequel
df_original = df[df.index % 2 == 0] # Excludes every 3rd row starting from 0
df_sequel   = df[df.index % 2 != 0] # Selects every 3rd raw starting from 0

In [17]:
df_original.reset_index(drop=True,inplace=True)

In [18]:
df_sequel.reset_index(drop=True,inplace=True)

In [19]:
df_original.head()

,Title,url,IMDB Score,Metacritic,Runtime (mins),Budget,Opening Weekend,Gross USA,Gross World,Release Date,Rating,Genres,Country
0,Spider-Man (2002),http://www.imdb.com/title/tt0145487/,7.3,73.0,121,139000000.0,114844116.0,407022860.0,8.250250e+08,2002-05-03,PG-13,Action Adventure Sci-Fi,USA
1,The Matrix (1999),http://www.imdb.com/title/tt0133093/,8.7,73.0,136,63000000.0,27788331.0,171479930.0,4.653438e+08,1999-03-31,R,Action Sci-Fi,USA
2,The Lord of the Rings: The Fellowship of the R...,http://www.imdb.com/title/tt0120737/,8.8,92.0,178,93000000.0,47211490.0,315544750.0,8.878328e+08,2001-12-19,PG-13,Action Adventure Drama,New Zealand
3,The Lord of the Rings: The Return of the King ...,http://www.imdb.com/title/tt0167260/,8.9,94.0,201,94000000.0,72629713.0,377845905.0,1.142219e+09,2003-12-17,PG-13,Adventure Drama Fantasy,New Zealand
4,The Hobbit: The Desolation of Smaug (2013),http://www.imdb.com/title/tt1170358/,7.8,66.0,161,225000000.0,73645197.0,258366855.0,9.583669e+08,2013-12-13,PG-13,Adventure Fantasy,New Zealand


In [20]:
df_sequel.head()

,Title,url,IMDB Score,Metacritic,Runtime (mins),Budget,Opening Weekend,Gross USA,Gross World,Release Date,Rating,Genres,Country
0,Spider-Man 2 (2004),http://www.imdb.com/title/tt0316654/,7.3,83.0,127,200000000.0,88156227.0,373585825.0,7.889765e+08,2004-06-30,PG-13,Action Adventure Sci-Fi,USA
1,The Matrix Reloaded (2003),http://www.imdb.com/title/tt0234215/,7.2,62.0,138,150000000.0,91774413.0,281576461.0,7.418465e+08,2003-05-15,R,Action Sci-Fi,USA
2,The Lord of the Rings: The Two Towers (2002),http://www.imdb.com/title/tt0167261/,8.7,87.0,179,94000000.0,62007528.0,342551365.0,9.512081e+08,2002-12-18,PG-13,Adventure Drama Fantasy,New Zealand
3,The Hobbit: An Unexpected Journey (2012),http://www.imdb.com/title/tt0903624/,7.8,58.0,169,180000000.0,84617303.0,303003568.0,1.017004e+09,2012-12-14,PG-13,Adventure Family Fantasy,New Zealand
4,The Hobbit: The Battle of the Five Armies (2014),http://www.imdb.com/title/tt2310332/,7.4,59.0,144,250000000.0,54724334.0,255119788.0,9.560198e+08,2014-12-17,PG-13,Adventure Fantasy,New Zealand


In [21]:
# Dump to new files
file_name_cleaned_org = 'movies_with_originals_only_final_cleaned.csv'
file_name_cleaned_seq = 'movies_with_sequels_only_final_cleaned.csv'

df_original.to_csv(data_path + file_name_cleaned_org, index=False)
df_sequel.to_csv(data_path + file_name_cleaned_seq, index=False)

In [22]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            539 non-null    object 
 1   url              539 non-null    object 
 2   IMDB Score       539 non-null    float64
 3   Metacritic       505 non-null    float64
 4   Runtime (mins)   539 non-null    int64  
 5   Budget           539 non-null    float64
 6   Opening Weekend  475 non-null    float64
 7   Gross USA        508 non-null    float64
 8   Gross World      539 non-null    float64
 9   Release Date     539 non-null    object 
 10  Rating           539 non-null    object 
 11  Genres           539 non-null    object 
 12  Country          539 non-null    object 
dtypes: float64(6), int64(1), object(6)
memory usage: 54.9+ KB


In [23]:
df_sequel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            539 non-null    object 
 1   url              539 non-null    object 
 2   IMDB Score       539 non-null    float64
 3   Metacritic       468 non-null    float64
 4   Runtime (mins)   539 non-null    int64  
 5   Budget           539 non-null    float64
 6   Opening Weekend  477 non-null    float64
 7   Gross USA        497 non-null    float64
 8   Gross World      539 non-null    float64
 9   Release Date     539 non-null    object 
 10  Rating           539 non-null    object 
 11  Genres           539 non-null    object 
 12  Country          539 non-null    object 
dtypes: float64(6), int64(1), object(6)
memory usage: 54.9+ KB


### Check for Negative Difference in Release Dates

A sanity check to see if there are negative days between original and release. This was done since I noticed there were delta days that either negative or really large. Function below will be used again in the next notebook.

In [24]:
# Let's check and see if there are any negative deltas between original and sequel
def diff_rel_dates(data_org,data_seq,delta_type):
    
    df_diff_days = (data_seq['Release Date'] - data_org['Release Date'])/np.timedelta64(1, delta_type)
    
    neg_diff_days = df_diff_days[df_diff_days < 0]
    
    if neg_diff_days is not None:
        print('No negative difference')
    else:
        print('There are negative time(s) between sequel and original. Please check!')

    return df_diff_days

In [25]:
diff_type = 'D'  # Let's use days as a difference

diff_days = diff_rel_dates(df_original,df_sequel,diff_type)


No negative difference


Great! There are no negative delta days between original and sequel.